## **Projeto Moretti** ##

In [1]:
import tkinter as tk
from tkinter import messagebox
from time import strftime, time
import sqlite3

# ---Cores do Projeto---
cor_principal = "#0066cc"
cor_secundaria = "#ffffff"
cor_tercearia = "#cccaca"

# --- Variáveis de Controle do Sistema ---
entrada_batida = False
saida_batida = False
ultima_entrada = None
ultima_saida = None
hora_ultima_saida = ""
TEMPO_ESPERA = 8 * 60 * 60
after_id = None

# Variável para guardar quem está logado agora
usuario_atual = {
    "nome": "",
    "email": ""
}

# --- Funções do Banco de Dados ---

def conectar():
    conexao = sqlite3.connect("ponto_senai.db")
    cursor = conexao.cursor()
    return conexao, cursor

def criar_tabelas():
    """Cria as tabelas de Usuários e Pontos se não existirem"""
    conexao, cursor = conectar()

    # [NOVO] Tabela de Usuários (Quem pode logar)
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        senha TEXT NOT NULL
    )''')

    # [NOVO] Tabela de Pontos atualizada com nome e email
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS pontos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tipo TEXT NOT NULL,
        data_visual TEXT NOT NULL,
        timestamp REAL NOT NULL,
        nome_usuario TEXT NOT NULL,
        email_usuario TEXT NOT NULL
    )''')

    conexao.commit()
    conexao.close()

def criar_usuarios_padrao():
    """Insere 2 funcionários se o banco estiver vazio"""
    conexao, cursor = conectar()

    # Verifica se já tem gente cadastrada
    cursor.execute("SELECT count(*) FROM usuarios")
    qtd = cursor.fetchone()[0]

    if qtd == 0:
        lista_usuarios = [
            ("Carlos Gerente", "carlos@moretti.com", "1234"),
            ("Ana Desenvolvedora", "ana@moretti.com", "1234")
        ]
        cursor.executemany("INSERT INTO usuarios (nome, email, senha) VALUES (?, ?, ?)", lista_usuarios)
        conexao.commit()
        print("Usuários padrão criados com sucesso!")

    conexao.close()

def buscar_historico_usuario():
    """Busca o histórico APENAS do usuário logado"""
    conexao, cursor = conectar()
    # Filtra pelo email do usuário logado
    cursor.execute("SELECT data_visual, tipo FROM pontos WHERE email_usuario = ?", (usuario_atual['email'],))
    dados = cursor.fetchall()
    conexao.close()
    return dados

def restaurar_estado_sistema():
    """Verifica o último ponto DO USUÁRIO LOGADO para saber se bate entrada ou saída"""
    global entrada_batida, saida_batida, ultima_entrada, ultima_saida, hora_ultima_saida

    # Reseta as variáveis ao trocar de usuário
    entrada_batida = False
    saida_batida = False
    ultima_entrada = None
    ultima_saida = None
    hora_ultima_saida = ""

    conexao, cursor = conectar()
    # [NOVO] Busca o último ponto DESTE email
    cursor.execute("SELECT tipo, timestamp, data_visual FROM pontos WHERE email_usuario = ? ORDER BY id DESC LIMIT 1", (usuario_atual['email'],))
    ultimo = cursor.fetchone()
    conexao.close()

    if ultimo:
        tipo_db, timestamp_db, data_visual_db = ultimo

        if tipo_db == "Entrada":
            entrada_batida = True
            saida_batida = False
            ultima_entrada = timestamp_db

        elif tipo_db == "Saída":
            entrada_batida = True
            saida_batida = True
            ultima_saida = timestamp_db
            hora_ultima_saida = data_visual_db.split(" ")[1]

    # Atualiza a interface com as novas infos
    atualizar_mensagem()

# --- Funções de Lógica ---

def validar_login():
    """ Verifica no banco se o email e senha existem"""
    email = entrada_email.get()
    senha = entrada_senha.get()

    conexao, cursor = conectar()
    cursor.execute("SELECT nome, email FROM usuarios WHERE email = ? AND senha = ?", (email, senha))
    usuario_encontrado = cursor.fetchone()
    conexao.close()

    if usuario_encontrado:
        # Salva na memória quem entrou
        usuario_atual['nome'] = usuario_encontrado[0]
        usuario_atual['email'] = usuario_encontrado[1]
        return True
    else:
        return False

def registrar_ponto(tipo):
    global entrada_batida, saida_batida, ultima_entrada, ultima_saida, hora_ultima_saida

    data_hora = strftime("%d/%m/%Y %H:%M:%S")
    agora = time()

    # Validações de lógica (igual antes)
    if tipo == "Saída" and not entrada_batida:
        messagebox.showwarning("Aviso", "Você ainda não bateu ponto de ENTRADA hoje!")
        return

    if tipo == "Entrada" and entrada_batida:
        if ultima_entrada and (agora - ultima_entrada < TEMPO_ESPERA):
             # Lógica de espera...
             messagebox.showwarning("Aviso", "Jornada em andamento.")
             return
        else:
             entrada_batida = False
             saida_batida = False

    elif tipo == "Saída" and saida_batida:
         if ultima_saida and (agora - ultima_saida < TEMPO_ESPERA):
             messagebox.showwarning("Aviso", "Você já saiu!")
             return

    # --- SALVAR NO BANCO ---
    try:
        conexao, cursor = conectar()
        # [NOVO] Agora salvamos também o nome e email do usuário atual
        cursor.execute('''
            INSERT INTO pontos (tipo, data_visual, timestamp, nome_usuario, email_usuario)
            VALUES (?, ?, ?, ?, ?)''',
            (tipo, data_hora, agora, usuario_atual['nome'], usuario_atual['email']))

        conexao.commit()
        conexao.close()

        # Atualiza visual
        if tipo == "Entrada":
            entrada_batida = True
            ultima_entrada = agora
            messagebox.showinfo("Moretti RH", f"Olá {usuario_atual['nome']}!\nEntrada registrada: {data_hora}")
        else:
            saida_batida = True
            ultima_saida = agora
            hora_ultima_saida = strftime("%H:%M:%S")
            messagebox.showinfo("Moretti RH", f"Até logo {usuario_atual['nome']}!\nSaída registrada: {data_hora}")

        atualizar_mensagem()

    except sqlite3.Error as erro:
        messagebox.showerror("Erro de Banco", f"Falha ao salvar: {erro}")

def ver_historico():
    dados_banco = buscar_historico_usuario()

    if not dados_banco:
        messagebox.showinfo("Histórico", "Nenhum ponto registrado para você.")
        return

    texto = "\n".join([f"{linha[0]} → {linha[1]}" for linha in dados_banco])

    janela_hist = tk.Toplevel(root)
    janela_hist.title(f"Histórico de {usuario_atual['nome']}")
    janela_hist.geometry("350x400")
    texto_box = tk.Text(janela_hist, wrap="word", font=("Arial", 10))
    texto_box.insert("1.0", texto)
    texto_box.config(state="disabled")
    texto_box.pack(expand=True, fill="both", padx=10, pady=10)

def atualizar_hora():
    global after_id
    hora = strftime("%H:%M:%S")
    if 'label_hora' in globals():
        label_hora.config(text=hora)

    atualizar_mensagem()
    after_id = root.after(1000, atualizar_hora)

def atualizar_mensagem():
    agora = time()
    mensagem = ""

    if entrada_batida and ultima_entrada:
        restante = TEMPO_ESPERA - (agora - ultima_entrada)
        if restante > 0 and saida_batida:
             mensagem += "Jornada finalizada. Aguardando próximo turno.\n"
        elif restante <= 0:
             mensagem += " Novo turno disponível!\n"

    if saida_batida and hora_ultima_saida:
        mensagem += f"Última saída: {hora_ultima_saida}\n"

    if 'label_msg' in globals():
        label_msg.config(text=mensagem.strip())

# --- Navegação ---

def mostrar_tela_login():
    global after_id
    if after_id:
        root.after_cancel(after_id)
        after_id = None

    if 'frame_ponto' in globals():
        frame_ponto.pack_forget()

    # Limpa os campos de login
    entrada_email.delete(0, tk.END)
    entrada_senha.delete(0, tk.END)

    frame_login.pack()
    root.title("Login - Moretti Systems")
    root.geometry(f"{largura_root}x{altura_root}+{pos_x}+{pos_y}")

def tentar_login():
    """Chamado pelo botão de login"""
    if validar_login():
        # Se o login for ok, carrega o estado DESSE usuário
        restaurar_estado_sistema()

        frame_login.pack_forget()

        # Atualiza o título com o nome da pessoa
        label_titulo_ponto.config(text=f"Olá, {usuario_atual['nome']}")

        root.title("Sistema de Ponto")
        root.geometry(f"330x400+{pos_x}+{pos_y}")
        frame_ponto.pack()
        atualizar_hora()
    else:
        messagebox.showerror("Erro", "Email ou senha inválidos!")

# --- Setup Inicial ---
root = tk.Tk()
root.resizable(False, False)

largura_root = 350
altura_root = 350 # Aumentei um pouco
largura_tela = root.winfo_screenwidth()
altura_tela = root.winfo_screenheight()
pos_x = (largura_tela // 2) - (largura_root // 2)
pos_y = (altura_tela // 2) - (altura_root // 2)

# --- Frame Login ---
frame_login = tk.Frame(root)
tk.Label(frame_login, text="Moretti Ponto", bg=cor_principal, fg="white", font=("arial", 25, "bold")).pack(fill="x", pady=(0, 20))

tk.Label(frame_login, text="Email Corporativo:", font=("arial", 11, "bold")).pack(anchor="w", padx=25)
entrada_email = tk.Entry(frame_login, relief="flat", width=30)
entrada_email.pack(pady=5)

tk.Label(frame_login, text="Senha:", font=("arial", 11, "bold")).pack(anchor="w", padx=25)
entrada_senha = tk.Entry(frame_login, width=30, relief="flat", show="*")
entrada_senha.pack(pady=5)

tk.Button(frame_login, text="Entrar", command=tentar_login, bg=cor_principal, fg="white", relief="flat", width=15).pack(pady=20)

root.bind('<Return>', lambda event: tentar_login())

# --- Frame Ponto ---
frame_ponto = tk.Frame(root)
# Label dinâmica que vai mostrar o nome do usuário
label_titulo_ponto = tk.Label(frame_ponto, text="Sistema de Ponto", font=("Arial", 14, "bold"))
label_titulo_ponto.pack(pady=10)

label_hora = tk.Label(frame_ponto, text="", font=("Arial", 14))
label_hora.pack()
label_msg = tk.Label(frame_ponto, text="", font=("Arial", 10), fg="gray")
label_msg.pack(pady=5)

tk.Button(frame_ponto, text="Bater Entrada", command=lambda: registrar_ponto("Entrada"), width=40, height=4, relief="flat", bg="#4CAF50", fg="white").pack(pady=5)
tk.Button(frame_ponto, text="Bater Saída", command=lambda: registrar_ponto("Saída"), width=40, height=4,relief="flat", bg="#f44336", fg="white").pack(pady=5)
tk.Button(frame_ponto, text="Ver Meu Histórico", command=ver_historico, width=20, height=2, bg="#2196F3",relief="flat", fg="white").pack(pady=5)
tk.Button(frame_ponto, text="Sair (Logout)", command=mostrar_tela_login, width=20, height=1, bg=cor_tercearia,relief="flat", fg="black").pack(pady=(10, 5))

# --- INICIALIZAÇÃO ---
criar_tabelas()      # Cria a estrutura
criar_usuarios_padrao()  # Cria o Carlos e a Ana
mostrar_tela_login() # Abre a tela

root.mainloop()

Usuários padrão criados com sucesso!
